In [1]:

import rtree, pygeos
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os
import folium

import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import numpy as np
import statsmodels.api as sm
from datetime import datetime, timedelta

ModuleNotFoundError: No module named 'pygeos'

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:

data_path = 'C:/Users/mhudgell/data/ais/'
telemetry_df = pd.read_parquet(data_path, engine='pyarrow')

NameError: name 'pd' is not defined

In [4]:
telemetry_df =  telemetry_df[['mmsi', 'acquisition_time', 'longitude', 'latitude', 'sog', 'cog']]

NameError: name 'telemetry_df' is not defined

In [ ]:
telemetry_df['sog'] = telemetry_df['sog'].astype(np.float32)

In [ ]:
local_df = telemetry_df[((telemetry_df['latitude'] > 51.85975370) &  (telemetry_df['latitude'] < 55.34311203)) &
                        ((telemetry_df['longitude'] > -7.33234434) & (telemetry_df['longitude'] < -2.45881019))]                


In [ ]:
print(len(local_df.index), len(telemetry_df.index))

In [ ]:
import movingpandas as mpd # needs python 3.9


In [ ]:
geometry = [Point(xy) for xy in zip(local_df.longitude, local_df.latitude)]
crs = {'init': 'epsg:4326'} #4326
telemetry_gdf = gpd.GeoDataFrame(local_df, crs=crs, geometry=geometry)
print(len(telemetry_gdf))
telemetry_gdf['t'] = pd.to_datetime(telemetry_gdf['acquisition_time'], format='%Y-%m-%d %H:%M:%S')
telemetry_gdf = telemetry_gdf.set_index('t')
#telemetry_gdf = [telemetry_gdf.sog>0]
print(len(telemetry_gdf))


In [ ]:
MIN_LENGTH = 100 # meters
traj_collection = mpd.TrajectoryCollection(gdf, 'mmsi', min_length=MIN_LENGTH)
print("Finished creating {} trajectories".format(len(traj_collection)))

In [ ]:
trips = mpd.ObservationGapSplitter(traj_collection).split(gap=timedelta(minutes=5))
print("Extracted {} individual trips from {} continuous vessel tracks".format(len(trips), len(traj_collection)))



In [ ]:
%%time
generalized = mpd.MinDistanceGeneralizer(trips).generalize(tolerance=100)

In [ ]:

%%time
aggregator = mpd.TrajectoryCollectionAggregator(generalized, max_distance=1000, min_distance=100, min_stop_duration=timedelta(minutes=5))



In [ ]:


pts = aggregator.get_significant_points_gdf()
clusters = aggregator.get_clusters_gdf()
( pts.hvplot(geo=True, tiles='OSM', frame_width=800) * 
  clusters.hvplot(geo=True, color='red') )


In [ ]:
flows = aggregator.get_flows_gdf()

In [ ]:
( aggregator_original.get_flows_gdf().hvplot(title='Original', geo=True, tiles='OSM', hover_cols=['weight'], line_width=dim('weight')*0.3, alpha=0.5, color='#1f77b3', frame_height=400, frame_width=400) * 
  aggregator_original.get_clusters_gdf().hvplot(geo=True, color='red', size='n') +
  flows.hvplot(title='Generalized', geo=True, tiles='OSM', hover_cols=['weight'], line_width=dim('weight')*0.3, alpha=0.5, color='#1f77b3', frame_height=400, frame_width=400) * 
  clusters.hvplot(geo=True, color='red', size='n') 
)